<h3>Bridger Bowl Snowfall Alert Notification Project</h3>

I'd like to receive a text every morning showing the snowfall details at Bridger Bowl from their website: https://bridgerbowl.com/weather/snow-report.

This notebook is to get the backend logic sorted out that will eventually end up on Lambda and scheduled to run each day to get an update. This is a basic version for now that does not include retries, exponential backoff, logging, or any logic to check when that the last website update was after the last notification, etc. -- it's just a basic rough draft to get it up and running.

I decided to use Selenium rather than BS4, since the snowfall data doesn't immediately appear on the website -- I can see it coming from some weather vendor service in the Network tab of Chrome page inspector. It's fairly slow to load (~1-3 seconds) when the page hasn't cached the data from a previous call, which is why BS4 wasn't able to load that data.

In [1]:
import requests
from bs4 import BeautifulSoup

def get_snowfall_data():
    url = "https://bridgerbowl.com/weather/snow-report"
    response = requests.get(url)
    response.raise_for_status()
    # print(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Modify the selector to match the site's structure
    new_snow = soup.find("div", class_="inline-flex flex-row text-2xl lg:text-3xl") # new snow
    print(new_snow)
    
    return new_snow

In [2]:
get_snowfall_data()

None


In [3]:
import requests
from bs4 import BeautifulSoup

def get_snowfall_data():
    url = "https://bridgerbowl.com/weather/snow-report" 
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the parent div with the class text-center
    snow_depth_section = soup.find("div", class_="text-center")
    if snow_depth_section:
        # Find the specific div containing the snowfall value
        base_snow_depth_element = snow_depth_section.find("div", class_="inline-flex flex-row text-2xl lg:text-3xl")
        if base_snow_depth_element:
            base_depth_value = snowfall_element.find("div").text.strip()
            return base_depth_value
        else:
            raise ValueError("Base snowfall value not found in the specified section")
    else:
        raise ValueError("Base depth section not found on the webpage")


In [5]:
get_snowfall_data()

ValueError: Base snowfall value not found in the specified section

<h3>Selenium over BS4</h3>

BS4 is better than Selenium if the content being scraped is static (already available in raw HTML when the page loads). However, the content on Bridger Bowl's website is dynamically generated via JavaScript after the page initially loads, so BS4 cannot access it because it only parses the raw HTML of the response from the server.

So, in this case, Selenium is the better option. The weather/snowfall data is not immediately loaded when the webpage is accessed. Selenium renders the page like a real browser, executing the JavaScript and exposing the final, updated DOM of the webpage (with the desired datapoints present).

Note that, in general, Selenium is better suited for complex interactions -- i.e., scenarios where you need to interact with the page (e.g., clicking buttons, scrolling, or waiting for elements to load). I am using `WebDriverWait` to ensure the specific snowfall data I am targeting appears on the page before scraping it.

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def get_snowfall_data_with_selenium():
    
    url = "https://bridgerbowl.com/weather/snow-report"
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:
        driver.get(url)

        # Sections to extract
        sections = ["Base Depth", "New Snow", "Recent Snow", "Seasonal Snowfall"]
        snowfall_data = {}

        wait = WebDriverWait(driver, 10)  # 10 seconds timeout

        for section in sections:
            try:
                # Wait for each section to appear
                section_element = wait.until(
                    EC.presence_of_element_located((By.XPATH, f"//div[h4[text()='{section}']]"))
                )

                # Locate the specific nested div and extract the value
                value_element = section_element.find_element(
                    By.CSS_SELECTOR, ".inline-flex.flex-row.text-2xl.lg\\:text-3xl div"
                )
                value = value_element.text.strip()
                snowfall_data[section] = str(value) + " inches"
                
            except Exception as e:
                print(f"Error extracting {section}: {e}")
                snowfall_data[section] = None  # Handle missing sections gracefully

            # Extract the "Last Updated" section
            try:
                last_updated_element = wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".text-sm.sm\\:text-md.md\\:text-base"))
                )
                last_updated_text = last_updated_element.text.strip()
                last_updated_text = last_updated_text.replace("Last Updated ", '')
                snowfall_data["Last Updated"] = last_updated_text
            except Exception as e:
                print(f"Error extracting Last Updated: {e}")
                snowfall_data["Last Updated"] = None

        return snowfall_data
        
    finally:
        driver.quit()

# Run the scraper
snowfall = get_snowfall_data_with_selenium()

# Format my message that will be sent out via SMS notification from AWS
message = (
    f"Bridger Bowl Snow Report\n\n"
    f"Last Updated: {snowfall['Last Updated']}\n\n"
    f"New Snow: {snowfall['New Snow']}\n"
    f"Recent Snow: {snowfall['Recent Snow']}\n"
    f"Base Depth: {snowfall['Base Depth']}\n"
    f"Seasonal Snowfall: {snowfall['Seasonal Snowfall']}\n"

)

# View the final message that will be sent out to subscribers
print(message)

Bridger Bowl Snow Report

Last Updated: Friday, January 10th, 2025 at 4:00pm

New Snow: 0 inches
Recent Snow: 0 inches
Base Depth: 52 inches
Seasonal Snowfall: 120 inches

